In [ ]:
# Import required libraries

import os
import h3
import rioxarray as rio
import xarray as xr
import numpy as np
import geopandas as gpd
import pandas as pd

from datetime import datetime
from glob import glob
from shapely.geometry import Polygon

In [ ]:
# Function definitions

def read_dem(dem):
    """
    Read and reproject a gdal-compatible raster file (e.g. DEM) in prep for creation of h3 hexagon delineation
    
    Parameters:
    dem (str): input DEM file
    
    Returns:
    Xarray data array representing the DEM 
    """
    
    # Open input elevation raster w/ rasterio
    i_data = rio.open_rasterio(dem, masked=True)
    
    # Reproject if necessary
    if i_data.rio.crs != "EPSG:4326":
        #print("Input CRS: " + str(i_data.rio.crs))
        target_crs = "EPSG:4326"
        o_data = i_data.rio.reproject(target_crs)
    else:
        o_data = i_data
    
    return o_data


def extract_valid_coords(data_arr):
    """
    Pull valid coordinates from an xarray data array
    
    Parameters:
    data_arr (object): Data array representing a DEM
    
    Returns:
    Arrays containing the coordinates of valid data 
    """
    
    # Define nodata mask
    valid_mask = (data_arr!=data_arr.rio.nodata)[0]
    valid_indices = np.argwhere(valid_mask.values)
    
    # Identify index locations of valid coordinate pairs
    lat_indices, lon_indices = valid_indices[:, 0], valid_indices[:, 1]
    valid_lat = data_arr.y[lat_indices].values
    valid_lon = data_arr.x[lon_indices].values
    
    return valid_lat, valid_lon

def create_h3_hex(h3_res, lats, lons):
    """
    Creates WKT geometry representing the h3 hexagons corresponding to spatial extent of input data file  
    
    Parameters:
    h3_res (str): Desired H3 level
    lats (str): array of latitude coordinates
    lons (str): array of longitude coordinates
    
    Returns:
    List of WKT geometries 
    """
    
    # Convert the latitude and longitude arrays to H3 indexes
    h3_indices = set(h3.geo_to_h3(lat, lon, h3_res) for lat, lon in zip(lats, lons))
    
    # Create hexagon geometry by converting H3 indices to polygons
    h3_polygons = [Polygon(h3.h3_to_geo_boundary(h, geo_json=True)) for h in h3_indices]
    
    return h3_polygons
  
def gdf2poly(h3_polys, surveyid, outfile):
    # Convert H3 polygons to a geodataframe, converting to a vector file format and write to disk
    gdf = gpd.GeoDataFrame({'survey_id': surveyid, 'geometry': h3_polys}, crs = "EPSG:4326")
    gdf.to_file(outfile)

In [ ]:
# User definition of variables

inpath = r"/mnt/c/Users/mike/glos_data/ocs_bag/rf_gtiff"
outpath = inpath
lof = glob(os.path.join(inpath, "*.tif"))
survname = "test"

In [ ]:
# Iterate through list of files defined above and create H3 hexagon outputs

for f in lof:
    print(f"processing file {f}") 
    dem = read_dem(f)
    coords = extract_valid_coords(dem)

    for hex_level in range(6, 11):
        output = os.path.join(outpath, f"{survname}_h3-{str(hex_level)}.geojson")
        h3_hex = create_h3_hex(hex_level, coords[0], coords[1])
        shp = gdf2shp(h3_hex, survname, output)

In [ ]:
# Aggregate multiple H3 hexagon vectors (surveys consisting of multiple files) and remove duplicative records
# A final aggregated vector file is written to disk

for i in range(6,11):
    print(i)
    
    gjson_lof = glob(os.path.join(outpath, f"*{i}.geojson"))
    gjson_gdf = []

    # Read each geojson file into a geodataframe
    for j in gjson_lof:
        gdf = gpd.read_file(j)
        gjson_gdf.append(gdf)
    
    # Create aggregated geodataframe
    comb_gdf = gpd.GeoDataFrame(pd.concat(gjson_gdf, ignore_index=True))

    # Deduplicate based on all columns
    deduplicated_gdf = comb_gdf.drop_duplicates().reset_index(drop=True)
   
    # Write to disk
    formatted_value = f"0{i}" if i < 10 else str(i) 
    deduplicated_gdf.to_file(os.path.join(outpath,"level" + formatted_value + "_dedup.geojson"))
    deduplicated_gdf.to_file(os.path.join(outpath,"level" + formatted_value + "_dedup.shp"))
    gjson_gdf = []

In [ ]:
# optional - plot on map

deduplicated_gdf.explore()